# Deploy Web application using Azure VM instance

In this exercise we will learn how to deploy our web app by creating a VM instance in Azure cloud infrastructure. As a prerequisite we assume you have created the [Azure cloud account](https://portal.azure.com). 

## Create a VM

![](images/vm/step_1.png)

Click on `Ubuntu Server -18.04 LTS`  and select `learn more`
![](images/vm/step_2.png)

## Select a default template
<br>
We are going to select the pre-set configuration for our VM server. Azure will pick the size of the VM and number of CPU's and RAM etc. You can change these configuration depending on your needs.

![](images/vm/step_3.png)

![](images/vm/step_4.png)

![](images/vm/step_5.png)

![](images/vm/step_6.png)

![](images/vm/step_7.png)

![](images/vm/step_8.png)

![](images/vm/step_9.png)

Since we are using an initial default template, we have already selected the size of our virtual machine. Hence we leave it default. Most of the deployment does not require the GPU to evaluate. But if your model requires a GPU to evaluate for some reason. Then you need to change VM instance and select the appropriate GPU options. But for now we use the default instance.

![](images/vm/step_10.png)

We can use ssh-key based login or password based login. Ideally we should use the ssh-key based login, which is more secure. Here we are using the `password` authentication method for demo purposes.
<br>
Also note that we need to keep port 22 open to be able to `ssh` to our virtual machine. We need to remember to close this port once we are done with deploying our application.
![](images/vm/step_11.png)

![](images/vm/step_12.png)

Once we hit the `create` button, we just need to wait for the VM to get composed. If you are seeing some snapshots similar to shown below, you should be good.
<br>
![](images/vm/step_13.png)

![](images/vm/step_14.png)

![](images/vm/step_15.png)

Now, our Virtual machine resource is ready for use.
<br>
![](images/vm/step_16.png)

Now we are ready with our Virtual machine. We need to note down our public ip address, so that we can configure our webpage to be accessible from the internet. You find your public ip address as shown below in the overview page of the dashboard.

![](images/vm/step_17.png)

## Configure the VM to launch our web app.

We can connect to our VM using linux terminal or Putty from windows. Here we use Azure CLI to connect with VM. You can run the same commands in linux /putty terminal. We do need the public ip of our VM to be able to connect.
<br>
![](images/vm/step_18.png)

![](images/vm/step_19.png)

## ssh connection to VM

We use the `username` and `public ip` to connect to our VM. Earlier, in the setup process, we configured `username` and `password` for `ssh` connections. In our example we created a user as `opencv`. We use the same user name to connect to our VM. Please note that, you need to provide the `username` you provided during configuration and the `public ip` address of your Virtual machine. `public ip` will be unique for each of the VM created in the Azure cloud.

**NOTE:** When you type password in the CLI, it will not show any characters such as `****`. Once you completed typing your password, hit enter key, that will login with your typed password. If it fails, re-try with correct password.

<br>

![](images/vm/step_20.png)

After providing the valid `ssh` username and password. You should be able to login to the Virtual machine which you have created. Here is a snapshot of how the successful login looks like.
<br>
![](images/vm/step_21.png)

## Configuring VM

update the existing software `sudo apt update`
<br>
![](images/vm/step_22.png)

Install python virtual environment dependencies. `sudo apt python3-venv`
![](images/vm/step_23.png)

create a virtual environment. We will create a virtual environment for our deployment named `deployment`
<br>
1. `sudo python3 -m venv deployment`
<br>

Activate the virutal environment.
<br>
2. `source deployment/bin/activate`
<br>

![](images/vm/step_24.png)

Clone the repository
<br>
1. `git clone https://github.com/bigvisionai/pytorch-web-app-deploy-azure.git`
2. `cd pytorch-web-app-deploy-azure`
<br>

![](images/vm/step_25.png)

Install the python dependencies to run our application.
`pip install -r requirements.txt`
<br>

![](images/vm/step_26.png)

You may observe some errors for `bdist_wheel`, but they are fine. You should observe the output as shown below.

![](images/vm/step_install_errors.png)

## Test run the application

Test the app to check if everything is loading as expected, without any errors. As part of test process we need to note down the port on which our application is running. It is important to note the `port` number as we need it to configure our network parameters in next step.


![](images/vm/step_27.png)

## Configure the network

Since our App will be running in the Virtual environment in the Azure cloud, we cannot access our application from the internet directly. We need to configure our network to be able to access it from external network. For that we need to add the port number of our application. i.e the ports on which our application is listening for request. In our case, from our previous step we noted that our application is listening on port `8000`. So we need to add `8000` port in the network configuration. Let us see how it is done.
1. Navigate to the Azure dash board.
2. Click on the `Networking` options on the left hand side of the panel.

![](images/vm/step_28.png)

Your networking panel should look as shown below. Click on `Add inbound port rule` button.

![](images/vm/step_29.png)

1. Add the port `8000` in the `Destination port ranges` field. 
2. Give it some name to be easily recognizable in the dash board. 
3. Click Add button to add the new rule.

![](images/vm/step_30.png)

Newly added rule should show up as shown below.

![](images/vm/step_31.png)

## Set environment variables.

Now we are almost ready to launch our app and be able to access it from the external network. But before that, remember our application expects some `Environment` variables, which we need to export. Please note that special care should be taken about your `SECRET_KEY` variable. As this varialbe will be used to encrypt the session and cookies related information of your web app.

![](images/vm/step_32.png)

## Run the app

Run the command `gunicorn --bind 0.0.0.0 -w 4 wsgi:app` in CLI. This will start the our application with 4 workers. You can change the number of workers to spawn depending on the load you expect on your server.
Now open your browser and a new tab. give `<your_public_ip>:8000` in the new tab. That should open the app!

![](images/vm/step_33.png)

![](images/vm/step_34.png)

![](images/vm/step_35.png)

![](images/vm/step_36.png)

# Robust deployment

Even though we were successfully able to deploy our application. There are few limitations or drawbacks with the approach being followed.
<br>
1. If you exit the session or close the terminal where you launched the application; Our application is no longer available!
2. It is a bit of inconvenience to give `port` number `8000` or `5000` at the end of the IP/domain name to land in to your app.
3. What if you want to give your domain name instead of ip address? How do we do that?
4. What if your VM restarts for some reason? But, your application doesn't launch automatically.
5. How will you Update your app? How will you keep up your app with the newer versions of Python and its dependencies?

We need to do some more engineering to achieve the above listed limitations with our current deployment strategy.
Let us address each one of them. Before we being let us login to our Virtual machine. 
`ssh <your_user_name>@<your_ip_address>`

## nginx and uwsgi 

We will use `nginx` as our webserver and `uwsgi` as hosting service to host our Flask application. 

### Installation
We will also install latest `python3.8` packages to demonstrate the package updation process.

```sh
$ sudo apt install python3.8 python3.8-dev python3-distutils uwsgi uwsgi-src uuid-dev libcap-dev libpcre3-dev python3-pip python3.8-venv nginx
```
<br>

Since we use python3.8, we need to build the matching plugin for `uwsgi`. For that we need the required uwsgi-plugin tools. Let us install the same.

```sh
$ sudo apt-get install uwsgi-plugin-python3
```

### Build the uwsgi-plugin
Run the following command to build the `uwsgi-plugin`. Let us name the plugin `python38` to know this plugin for the `python3.8` version for our future references.

```sh
$ export PYTHON=python3.8
$ uwsgi --build-plugin "/usr/src/uwsgi/plugins/python python38"
```
Above command will generate `python38_plugin.so` file in the current working directory. We need to move it to appropriate location. And give proper permissions to be able to access it.

```sh
$ sudo mv python38_plugin.so /usr/lib/uwsgi/plugins/python38_plugin.so
$ sudo chmod 666 /usr/lib/uwsgi/plugins/python38_plugin.so
```

### Create a new virtual environment with python3.8
<br>
There is nothing special about this. Except we create a new virtual environment with `python3.8` and install our project dependencies.
<br>

```sh
$ cd 
$ python3.8 -m venv python38
$ source python38/bin/activate
$ git clone https://github.com/bigvisionai/pytorch-web-app-deploy-azure.git
$ cd pytorch-web-app-deploy-azure
$ pip install -r requirements.txt
```
Above steps should install the project dependencies.

### Configure our app with uwsgi configuration.
<br>

We will create a `torchapp.ini` configuration file. We use this newly created file to inform `uwsgi` to pick the configurations required for our application using `torchapp.ini` file.

**NOTE:** we already have `torchapp.ini` file in our folder. We just need to change the `chdir = /home/opencv/pytorch-web-app-deploy-azure/` depending on the where you cloned the repo. 

```sh
# torchapp.ini 
[uwsgi]
chdir = /home/opencv/pytorch-web-app-deploy-azure/ 
module = wsgi:app

processes = 4
threads = 2
plugin = python38
virtualenv = /home/opencv/python38 

master = true
socket = torchapp.sock
chmod-socket = 666
vacuum = true

die-on-term = true

env = FLASK_APP=wsgi.py
env = FLASK_DEBUG=0
env = APP_CONFIG_FILE=config.py
env = UPLOADED_PHOTOS_DEST=/tmp/images/
env = SECRET_KEY="Please_enter_your_top_secerte_key"

```

Let us get to some details of `torchapp.ini` file

We change our root directory as where you have cloned the repository. In this example we have cloned the repo in `/home/opencv/` directory and `git clone` created a directory `pytorch-web-app-deploy-azure` and hence I am mentioning the path as `/home/opencv/pytorch-web-app-deploy-azure/`. You need to change it according to your directory. `uwsgi` will get the relative path for other resources.
```sh
chdir = /home/opencv/pytorch-web-app-deploy-azure/ 
module = wsgi:app
```
And our entry point is present in `wsgi.py`, so we let uwsgi know the same with `module = wsgi:app`

```sh
processes = 4
threads = 2
plugin = python38
virtualenv = /home/opencv/python38 
```
As earlier we inform the `uwsgi` to spawn 4 process with 2 threads each. We provide the `uwsgi-python-plugin` and the path of our virtual environment.

```sh
master = true
socket = torchapp.sock
chmod-socket = 666
vacuum = true

die-on-term = true
```
By setting the `master=true` we will let the uwsgi to run in master mode. When running with the master process mode, the uWSGI server can be gracefully restarted without closing the main sockets.

If you remember earlier we were mentioning the port number as `8000` or `5000` etc. Instead of opening a port directly like that we create a UNIX socket named `torch.sock` in the `chdir = /home/opencv/pytorch-web-app-deploy-azure/` directory. This will act as a proxy port to our nginx server. We can configure the nginx server to listen on this UNIX port to route our requests to reach our web pages. We will revisit the `socket = torchapp.sock` options when doing nginx configuration shortly.

We need to set permission `666` on the socket. We change the permissions by `chmod-socket = 666` in configuration file.
`vacuum = true` tells the uwsgi process to remove and clean up the opened sockets on exit, gracefully.
`die-on-term = true`  will let you kill the application by using `ctrl+c` option.

If we re-collect from our `config.py` file, we read some of the environment variables in our app. We set the required environment variables as follows.

```sh
env = FLASK_APP=wsgi.py
env = FLASK_DEBUG=0
env = APP_CONFIG_FILE=config.py
env = UPLOADED_PHOTOS_DEST=/tmp/images/
env = SECRET_KEY="Please_enter_your_top_secerte_key"
```
Again remember to change the `SECRET_KEY="Please_enter_your_top_secerte_key"` to your keys.

we can test if the configurations are working fine or not by running the following command.

`uwsgi torchapp.ini`. This should show up the familiar logs. One thing to note is you should be able to see `torchapp.sock` file being created when `uwsgi` is running. You can verify the same by logging in VM on a different terminal by changing in to your application directory. In our example `cd  /home/opencv/pytorch-web-app-deploy-azure/ ` and `ls torchapp.sock`.
<br>

You should also notice that when you kill the application using `ctrl+c` option, you should no longer see the file `torchapp.sock` file in your application directory.

With these steps we are one step closer to our goal

## Configure nginx

We need to have a configuration file to point nginx server to look for. Let us create one.

```sh
$ sudo vim /etc/nginx/sites-available/torchapp.conf
```

Add the following lines in to your `/etc/nginx/sites-available/torchapp.conf` file

```sh
server {
    listen 80;
    server_name your_server_ip_or_domain_name; 

    location / {
        include uwsgi_params;
        uwsgi_pass unix:/home/opencv/pytorch-web-app-deploy-azure/torchapp.sock;
    }
}
```

`listen 80` tells the nginx server to listen on port `80` this is the default port for http requests. This will help us to remove the ugly `http://<your_ip_address>:8000` url and make it simple `http://<your_ip_address>`

`server_name your_server_ip_or_domain_name;` here you can give your domain name or ip address. For example if your Virtual machine's public ip is `123.1.1.123` then you provide the server name as `server_name 123.1.1.123;`

**NOTE:** Do not forget to change the `server_name your_server_ip_or_domain_name`.

<br>

Suppose if you have a domain name `www.example.com` then your server name looks as `server_name www.example.com`

We need to tell the nginx to listen on the proxy port and route the traffic will arrive on port `80` to our application. We can achieve the same as follows.

```sh
    location / {
        include uwsgi_params;
        uwsgi_pass unix:/home/opencv/pytorch-web-app-deploy-azure/torchapp.sock;
    }
```
It is important to note the line `uwsgi_pass unix:/home/opencv/pytorch-web-app-deploy-azure/torchapp.sock`. Here we are instructing the nginx server to look for the port `torchapp.sock` present in our application directory. 

**NOTE:** You need to change the path `unix:/home/opencv/pytorch-web-app-deploy-azure/torchapp.sock` according to your path where you cloned the repository.

If for some reason uwsgi cannot create the `torchapp.sock` file or nginx doesn't find the socket file `torchapp.sock` then you get the error as shown below.

![](images/502_bad_gateway_error.png)

### Test the nginx configuration file

We need to make sure our configuration file doesn't have any syntax errors. We can test the configuration by running the following command

```sh
$ sudo nginx -T
```
Suppose we made some syntax error in our script. above command will help us to identify the problem in our script by throwing some error as shown below. 
```sh
nginx: [emerg] directive "server_name" is not terminated by ";" in /etc/nginx/sites-enabled/torchapp.conf:5
nginx: configuration file /etc/nginx/nginx.conf test failed 
```

If there are no errors in your configuration file then `sudo nginx -T` command will puts lots of logs on the console with, different configuration file details. Small chunk of log is shown below.

```sh
....
...
#server {
#	listen 80;
#	listen [::]:80;
#
#	server_name example.com;
#
#	root /var/www/example.com;
#	index index.html;
#
#	location / {
#		try_files $uri $uri/ =404;
#	}
#}

# configuration file /etc/nginx/sites-enabled/torchapp.conf:
server {
    listen 80;
    server_name 123.1.1.123; 

    location / {
        include uwsgi_params;
        uwsgi_pass unix:/home/opencv/pytorch-web-app-deploy-azure/torchapp.sock;
    }
}

# configuration file /etc/nginx/uwsgi_params:

uwsgi_param  QUERY_STRING       $query_string;
uwsgi_param  REQUEST_METHOD     $request_method;
uwsgi_param  CONTENT_TYPE       $content_type;
uwsgi_param  CONTENT_LENGTH     $content_length;
....

```

We can observe in the above log that our configuration file doesn't have any syntax errors. And we can proceed with our configuration steps. Unfortunately above command doesn't validate if the value provided in the fields are valid or not. Say for example you provided incorrect ip address or incorrect domain name. It doesn't validate the same. Hence a special care needs to be taken while entering those values.

Now we need to inform the nginx service that our application is available and enabled. Here is how we do that.
```sh
$ sudo ln -s /etc/nginx/sites-available/torchapp.conf /etc/nginx/sites-enabled/torchapp.conf
```
now we need to restart the nginx service to pick up the latest changes.

```sh
$ sudo service nginx restart
```

### Configure the port of VM 

If we recollect, we did configure the VM network for port `8000`

![](images/vm/step_30.png)

Now we need to change the value of the port. We want it to be `80` instead of `8000`. Let us edit the same
![](images/port_reconfig.png)

Now we have fixed the `port` issue and we have a way to provide our custom domain name. We can check if everything is working fine. 

**Note:** The order of commands run below is important

```sh
# change to the directory where your application code is cloned.
# in our illustration we have our code in the path /home/opencv/pytorch-web-app-deploy-azure
$ cd /home/opencv/pytorch-web-app-deploy-azure
$ uwsgi torchapp.ini
$ sudo service nginx restart
```

Now we can open the browser and open a new tab and provide our `public ip` or `domain name` to see our friendly web app home page.

## Fixing the Reboot issue.

So far, we have not done anything to fix the reboot issue or session expire issues of our web app. Let us see how to fix the same.
<br>
There are many approaches we can follow to achieve the same. But we stick with the simplest and reliable option.
We will use the `uwsgi`'s `emperor` option to get the work done. This is useful because

1. It is simple to use.
2. If we want to deploy multiple Web applications in a single VM, we can achive the same quite easily.

### Step 1.

Let `uwsgi` know what all the available applications in the the VM.

```sh
$ sudo cp /home/opencv/pytorch-web-app-deploy-azure/torchapp.ini /etc/uwsgi/apps-available/torchapp.ini
```
Enable the application's to be controlled by `uwsgi`. We can achieve the same by creating a soft link to the configuration file under `apps-enabled` directory.

```sh
$ sudo ln -s /etc/uwsgi/apps-available/torchapp.ini /etc/uwsgi/apps-enabled/torchapp.ini
```

### Step 2

We use the `/etc/rc.local` file to bring up our application after each reboot. If the file doesn't exist, then just create a new file using the command `sudo touch /etc/rc.local`.
Copy and paste the below contents in to the file `/etc/rc.local`. Save and exit. 

```sh
#!/bin/sh -e
uwsgi --emperor /etc/uwsgi/apps-enabled/
sudo service nginx restart
```
Make sure the file is not empty, by running the command `cat /etc/rc.local`.
<br>

We need to change the permissions of the `/etc/rc.local` file by running the below command.

```sh
$ sudo chmod a+x /etc/rc.local
```

Thats it! Now we are all set. Go ahead and reboot your virtual machine. After few mins you can directly go to the browser and give your `ip address` or `domain name` in the browser. You can see that page gets loaded as expected.



## Debugging

No matter how hard you try to follow steps, we always end up with some or the other issues. Here are some of the debugging tips to help you get it corrected. It is good idea to go through the complete Debugging sections, to know what kind of issues can arise and how to identify and fix the same.

### page not found error

Quite often we encounter the page not found error.

![](images/error_loading_page.png)

It means most probably you don't have the `nginx` server running. Here is how we know the status of the nginx service. 

```sh
$ sudo service nginx status
```

Here is a sample healthy `nginx` server log.

```sh
nginx.service - A high performance web server and a reverse proxy server
   Loaded: loaded (/lib/systemd/system/nginx.service; enabled; vendor preset: enabled)
   Active: active (running) since Thu 2020-05-07 07:35:38 UTC; 5s ago
     Docs: man:nginx(8)
  Process: 2849 ExecStop=/sbin/start-stop-daemon --quiet --stop --retry QUIT/5 --pidfile /run/nginx.pid (code=exited, status=0/SUCCESS)
  Process: 2866 ExecStart=/usr/sbin/nginx -g daemon on; master_process on; (code=exited, status=0/SUCCESS)
  Process: 2850 ExecStartPre=/usr/sbin/nginx -t -q -g daemon on; master_process on; (code=exited, status=0/SUCCESS)
...
...
```

Note the line `Active: active (running) since ....` in the above log. Which informs that nginx is running and it is active.

If there is an error then. Most probably something went wrong with your configuration file which we created `/etc/nginx/sites-available/torchapp.conf`

* If there is an error in the `nginx` service then, we need to check our configuration. Run the command `sudo nginx -T`. Hopefully we find some errors in our config file we can fix it, by reading the nginx configuration steps again.

* If above two steps are fine with you. Then most probably forgot to enable the port `80` in your network configuration or gave a different port number in your `/etc/nginx/sites-available/torchapp.conf`  and forgot to add it in your network configuration.

## Page routing to default nginx page.

If you encounter a page as shown below.
![](images/default_nginx_page.png)

It is some good news, your nginx server is working as expected and you have enabled the correct port in VM's network configurations. So what could go wrong?

This could basically mean your `uwsgi` could have some problem. Let us see how to debug the same.

* Check if your `uwsgi` service is running. If your uwsgi is running then you should see  log something similar to as shown below. 
```sh
ps -A | grep "uwsgi"
  1096 ?        00:00:00 uwsgi
  1180 ?        00:00:01 uwsgi-core
  1569 ?        00:00:00 uwsgi-core
  1570 ?        00:00:00 uwsgi-core
  1571 ?        00:00:00 uwsgi-core
  1575 ?        00:00:00 uwsgi-core
```
* If you see the `uwsgi` is running and have the `torchapp.socket` file is created. Then just restart the `nginx` service `sudo service nginx restart`. Check reloading the page in browser and see if issue is resolved. If not continue reading next debug step.

* If there is no output for the `ps -A | grep "uwsgi"` in your terminal. Then your uwsgi service is not running. 
    * This could mean there could be something wrong with your `torchapp.ini` configuration file. Best thing to do is run the command `uwsgi torchapp.ini` manually and check for errors. Make sure `torchapp.socket` file has been created your `chdir = /home/opencv/pytorch-web-app-deploy-azure/` directory. If you don't find it, that is a problem.
    * If the above step is working as expected without any errors then the `/home/opencv/pytorch-web-app-deploy-azure/torchapp.ini` and `/etc/uwsgi/apps-available/torchapp.ini` doesn't match. try to run the app with the configuration `uwsgi /etc/uwsgi/apps-available/torchapp.ini` and check if `torchapp.socket` is created.



## 502 bad Gateway error

If you find the error as show below, there are couple of things which could have go wrong.

![](images/502_bad_gateway_error.png)

*  Make sure `torchapp.socket` file has been created your `chdir = /home/opencv/pytorch-web-app-deploy-azure/` directory. 
* If the file is present, that means `nginx` is not able to find the socket file `torchapp.socket`. Simple solution is to restart the `nginx` service `sudo service nginx restart` and try reloading page.


If you still have some problem, then it is time to ask for help in forums! :)